In [118]:
!pip install nvidia-ml-py3

!pip install --upgrade tensorflow==2.7.0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |█████████████████████           | 319.8 MB 124.9 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████▎   | 432.9 MB 109.5 MB/s eta 0:00:01

     |████████████████████████████▊   | 438.8 MB 109.5 MB/s eta 0:00:01

     |█████████████████████████████   | 444.8 MB 109.5 MB/s eta 0:00:01

...

     |███████████████████████████████ | 474.6 MB 109.8 MB/s eta 0:00:01

     |███████████████████████████████▍| 480.6 MB 109.8 MB/s eta 0:00:01

     |███████████████████████████████▉| 486.5 MB 109.8 MB/s eta 0:00:01

     |████████████████████████████████| 489.6 MB 109.8 MB/s eta 0:00:01

     |████████████████████████████████| 489.6 MB 47 kB/s               
     |████████████████████████████████| 1.3 MB 136.2 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [23]:
import os
import pathlib
import cv2

import numpy as np
import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras.layers import Layer, Input, Dense, Flatten, Lambda

from sklearn.model_selection import train_test_split

import random
import shutil

## GPU Setup

In [114]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
# Check availability of GPU
import nvidia_smi 
import os 

GPU = 1
os.environ["CUDA_VISIBLE_DEVICES"] = '{}'.format(GPU)

gpus = tf.config.experimental.list_physical_devices('GPU')
print('GPU is', 'available' if gpus else 'NOT AVAILABLE')
print('Number GPUs Available: ', len(gpus))


# Restrict TensorFlow to only use the MENTIONED GPU
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print('{} Physical GPUs, Logical GPU'.format(len(gpus), len(logical_gpus)))
        print('Memory growth set to: {} '.format(str(tf.config.experimental.get_memory_growth(gpus[0]))))
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

GPU is available
Number GPUs Available:  1
1 Physical GPUs, Logical GPU
Memory growth set to: True 


2022-02-28 13:40:01.977520: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-28 13:40:02.893026: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 40277 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:65:00.0, compute capability: 8.6


In [3]:
for gpu in gpus:
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
# Initiate NVIDIA-SMI
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(GPU)


# Get GPU memory usage
def print_GPU_usage():
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    print("Total memory: {}".format(info.total // (1024 * 1024) ))
    print("Used memory: {}".format(info.used // (1024 * 1024) ))

In [6]:
print_GPU_usage()

Total memory: 46068
Used memory: 4254


## Set up Directories

In [69]:
# # for each of the dirs, create anchor, positive, & negative
# for directory in os.listdir('before_augmentation'):
#     POS_PATH = os.path.join('siamese_data', directory, 'positive')
#     NEG_PATH = os.path.join('siamese_data', directory, 'negative')
#     ANC_PATH = os.path.join('siamese_data', directory, 'anchor')
#     os.makedirs(POS_PATH)
#     os.makedirs(NEG_PATH)
#     os.makedirs(ANC_PATH)

In [71]:
# # copy images to each of anchor directories
# for directory in os.listdir('before_augmentation'):
#     for file in os.listdir(os.path.join('before_augmentation', directory)):
#         EX_PATH = os.path.join('before_augmentation', directory, file)
#         NEW_PATH = os.path.join('siamese_data', directory, 'anchor', file)
#         shutil.copy(EX_PATH, NEW_PATH)

In [72]:
# # copy images to positive directory in the corresponding cluster
# for directory in os.listdir('after_augmentation'):
#     for file in os.listdir(os.path.join('after_augmentation', directory)):
#         EX_PATH = os.path.join('after_augmentation', directory, file)
#         NEW_PATH = os.path.join('siamese_data', directory, 'positive', file)
#         shutil.copy(EX_PATH, NEW_PATH)

In [75]:
# # randomly select 30 images from other clusters and move them to negative directory
# for directory in os.listdir('siamese_data'):
#     for other_directory in os.listdir('siamese_data'):
#         if other_directory != directory:
#             samples = random.sample(os.listdir(os.path.join('siamese_data', other_directory, 'positive')), 30)
#             for file in samples:
#                 EX_PATH = os.path.join('siamese_data', other_directory, 'positive', file)
#                 NEW_PATH = os.path.join('siamese_data', directory, 'negative', file)
#                 shutil.copy(EX_PATH, NEW_PATH)

## Preprocess Dataset

In [7]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (224,224))
    img = img / 255.0
    return img

In [8]:
arr_len = len([f for f in os.listdir('siamese_data')])
num_files = []
anchors, positives, negatives = [None] * arr_len, [None] * arr_len, [None] * arr_len
index = 0

# determine number of files in each anchor directory (max number of files we can take)
for directory in sorted(os.listdir('siamese_data')):
    num_files.append(len([f for f in os.listdir(os.path.join('siamese_data', directory, 'anchor'))]))

In [9]:
# store in array of anchor/positive/negative datasets
for directory in sorted(os.listdir('siamese_data')):
    numFiles = 300 if num_files[index] > 300 else num_files[index]
    anchors[index] = tf.data.Dataset.list_files(os.path.join('siamese_data', directory, 'anchor') + '/*.jpg').take(numFiles)
    positives[index] = tf.data.Dataset.list_files(os.path.join('siamese_data', directory, 'positive') + '/*.jpg').take(numFiles)
    negatives[index] = tf.data.Dataset.list_files(os.path.join('siamese_data', directory, 'negative') + '/*.jpg').take(numFiles)
    index += 1

In [10]:
data = [None] * arr_len
index = 0

for _ in range(arr_len):
    positive_temp = tf.data.Dataset.zip((anchors[index], positives[index], tf.data.Dataset.from_tensor_slices(tf.ones(len(anchors[index])))))
    negative_temp = tf.data.Dataset.zip((anchors[index], negatives[index], tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchors[index])))))
    data[index] = positive_temp.concatenate(negative_temp)
    index += 1

In [11]:
# 0-26 dirs for test_data and 27-32 dirs for train_data
test_data_temp = data[len(data) - 6:]
train_data_temp = data[:len(data) - 6]

In [12]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [13]:
# combine array of train_data into a single dataset
train_data = train_data_temp[0]
for i in range(1, len(train_data_temp)):
    train_data = train_data.concatenate(train_data_temp[i])

In [14]:
# build dataloader pipeline
train_data = train_data.map(preprocess_twin)
train_data = train_data.cache()
train_data = train_data.shuffle(buffer_size=1024)

In [15]:
len(train_data)

2484

In [16]:
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [17]:
len(train_data)

156

## Build Model w/ Transfer Learning

In [114]:
def make_base_model(inputs):
    base_model = keras.applications.VGG16(
        weights='imagenet',
        input_shape=(224,224,3),
        include_top=False
    )
    
    base_model.trainable = False
    x = base_model(inputs, training=False)
    
    # Convert features of shape `base_model.output_shape[1:]` to vectors
    x = keras.layers.GlobalAveragePooling2D()(x)
    
    # f_layer = Flatten()(x)
    outputs = Dense(4096, activation='sigmoid')(x)
#     d_layer = Dense(256, activation=None)(x)
#     outputs = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(d_layer) # L2 normalize embeddings
    
    return keras.Model(inputs, outputs)

In [105]:
# Siamese L1 Distance class
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

## Siamese Model

In [106]:
def make_siamese_model():
    
    # Anchor image input
    inp_image = Input(name='input_img', shape=(224,224,3))
    
    # Validation image input
    val_image = Input(name='validation_img', shape=(224,224,3))
    
    siamese_layer = L1Dist()
    inp_embedding = make_base_model(inp_image)
    val_embedding = make_base_model(val_image)
    
    distances = siamese_layer(inp_embedding(inp_image), val_embedding(val_image))
    
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return keras.Model(inputs=[inp_image, val_image], outputs=classifier, name='SiaemeseNetwork')

In [107]:
siamese_model = make_siamese_model()

In [108]:
siamese_model.summary()

Model: "SiaemeseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_2 (Functional)           (None, 4096)         16815936    ['input_img[0][0]']              
                                                                                    

## Training

In [32]:
!pip3 install -q -U tensorflow-addons

In [34]:
import tensorflow_addons as tfa

In [35]:
# siamese_model.compile(
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     loss=tfa.losses.TripletSemiHardLoss())

In [37]:
# history = siamese_model.fit(
#     train_data,
#     epochs=5)

Epoch 1/5


ValueError: in user code:

    File "/home/tf/.local/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/tf/.local/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/tf/.local/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/tf/.local/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/tf/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/tf/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 199, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" e

In [109]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
# triplet_loss = tfa.losses.TripletSemiHardLoss()
# contrastive_loss = tfa.losses.contrastive_loss()
opt = tf.keras.optimizers.Adam(1e-4)

In [39]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [110]:
@tf.function
def train_step(batch):
    
    # record all operations
    with tf.GradientTape() as tape:
        # get anchor and positive/negative image
        X = batch[:2]
        # get the label
        y = batch[2]
        
        # forward pass
        yhat = siamese_model(X, training=True)
        # calculate loss
        loss = binary_cross_loss(y, yhat)
#         loss = triplet_loss(y, yhat)
#         loss = tfa.losses.contrastive_loss(y, yhat)
    print(loss)
        
    # calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

In [111]:
def train(data, EPOCHS):
    # loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
    
        # loop through each batch
        for idx, batch in enumerate(data):
            # run train step
            train_step(batch)
            progbar.update(idx+1)

        # save checkpoints
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [112]:
EPOCHS = 50

train(train_data, EPOCHS)


 Epoch 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
156/156 [==============================] - 6s 32ms/step

 Epoch 2/50
156/156 [==============================] - 5s 30ms/step

 Epoch 3/50
156/156 [==============================] - 5s 30ms/step

 Epoch 4/50
156/156 [==============================] - 5s 30ms/step

 Epoch 5/50
156/156 [==============================] - 5s 30ms/step

 Epoch 6/50
156/156 [==============================] - 5s 30ms/step

 Epoch 7/50
156/156 [==============================] - 5s 30ms/step

 Epoch 8/50
156/156 [==============================] - 5s 30ms/step

 Epoch 9/50
156/156 [==============================] - 5s 30ms/step

 Epoch 10/50
156/156 [==============================] - 5s 30ms/step

 Epoch 11/50
156/156 [==============================] - 5s 30ms/step

 Epoch 12/50
156/156 [==============================] - 5s 30ms/step

 Epoch 13/50
1

### Evaulate Model

In [122]:
from tensorflow.keras.metrics import Precision, Recall

In [123]:
len(test_data_temp)

6

In [124]:
# combine array of train_data into a single dataset
test_data = test_data_temp[0]
for i in range(1, len(test_data_temp)):
    test_data = test_data.concatenate(test_data_temp[i])

In [125]:
samples = test_data.as_numpy_iterator()

In [126]:
# build dataloader pipeline
test_data = test_data.map(preprocess_twin)
test_data = test_data.cache()
test_data = test_data.shuffle(buffer_size=1024)

In [127]:
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [136]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [137]:
y_true

array([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.],
      dtype=float32)

In [138]:
y_hat = siamese_model.predict([test_input, test_val])

In [139]:
m = Recall()

m.update_state(y_true, y_hat)

m.result().numpy()

0.2